# Rotations and Transformations
- 3D rotations, Quaternions, Euler angles
- Real examples: Robotics, Computer graphics, IMU sensors

In [1]:
import numpy as np
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt
print('Rotation and transformation module loaded')

Rotation and transformation module loaded


## Rotation Representations
**Multiple representations**:
- **Matrix**: 3×3 orthogonal matrix
- **Quaternion**: 4D unit vector (w, x, y, z)
- **Euler angles**: Roll, pitch, yaw (order matters!)
- **Axis-angle**: Rotation axis + angle

**scipy.spatial.transform.Rotation**: Unified interface

In [2]:
# Create rotation: 90° around Z-axis
angle_deg = 90
print(f'Creating {angle_deg}° rotation around Z-axis\n')

# From Euler angles (intrinsic, XYZ order)
r = R.from_euler('z', angle_deg, degrees=True)

print('Rotation representations:')

# 1. Rotation matrix
matrix = r.as_matrix()
print(f'1. Rotation matrix (3×3):\n{matrix}\n')

# 2. Quaternion
quat = r.as_quat()  # [x, y, z, w]
print(f'2. Quaternion [x,y,z,w]: {quat}\n')

# 3. Euler angles
euler = r.as_euler('xyz', degrees=True)
print(f'3. Euler angles (XYZ): {euler}°\n')

# 4. Axis-angle
rotvec = r.as_rotvec()
axis = rotvec / np.linalg.norm(rotvec)
angle_rad = np.linalg.norm(rotvec)
print(f'4. Axis-angle:')
print(f'   Axis: {axis}')
print(f'   Angle: {np.degrees(angle_rad):.1f}°')

Creating 90° rotation around Z-axis

Rotation representations:
1. Rotation matrix (3×3):
[[ 2.22044605e-16 -1.00000000e+00  0.00000000e+00]
 [ 1.00000000e+00  2.22044605e-16  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]

2. Quaternion [x,y,z,w]: [0.         0.         0.70710678 0.70710678]

3. Euler angles (XYZ): [ 0.  0. 90.]°

4. Axis-angle:
   Axis: [0. 0. 1.]
   Angle: 90.0°


## Applying Rotations
Rotate vectors/points in 3D space

In [3]:
# Vector to rotate
vector = np.array([1, 0, 0])  # Point along X-axis

print(f'Original vector: {vector}')

# Apply rotation (90° around Z)
rotated = r.apply(vector)

print(f'After 90° Z-rotation: {rotated}')
print(f'Expected: [0, 1, 0] (X→Y)\n')

# Multiple vectors
vectors = np.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1]
])
rotated_all = r.apply(vectors)
print(f'Rotating multiple vectors:')
for i, (orig, rot) in enumerate(zip(vectors, rotated_all)):
    print(f'  {orig} → {rot}')

Original vector: [1 0 0]
After 90° Z-rotation: [2.22044605e-16 1.00000000e+00 0.00000000e+00]
Expected: [0, 1, 0] (X→Y)

Rotating multiple vectors:
  [1 0 0] → [2.22044605e-16 1.00000000e+00 0.00000000e+00]
  [0 1 0] → [-1.00000000e+00  2.22044605e-16  0.00000000e+00]
  [0 0 1] → [0. 0. 1.]


## Composing Rotations
Combine multiple rotations via multiplication
**Order matters!** R1 * R2 ≠ R2 * R1

In [4]:
# Two rotations
r1 = R.from_euler('z', 45, degrees=True)  # 45° around Z
r2 = R.from_euler('x', 30, degrees=True)  # 30° around X

print('Composing rotations:\n')

# Composition (r2 applied first, then r1)
r_combined = r1 * r2

print('Combined rotation (r1 * r2):')
print(f'  Euler: {r_combined.as_euler("xyz", degrees=True)}°\n')

# Apply to vector
vec = np.array([1, 0, 0])
result1 = r_combined.apply(vec)
result2 = r1.apply(r2.apply(vec))  # Equivalent

print(f'Vector {vec} after combined rotation: {result1}')
print(f'Same as r1(r2(vec)): {result2}')
print(f'Match: {np.allclose(result1, result2)}')

Composing rotations:

Combined rotation (r1 * r2):
  Euler: [30.  0. 45.]°

Vector [1 0 0] after combined rotation: [ 7.07106781e-01  7.07106781e-01 -2.77555756e-17]
Same as r1(r2(vec)): [0.70710678 0.70710678 0.        ]
Match: True


## Real Example: Drone Orientation
Convert IMU sensor data (quaternions) to Euler angles
Applications: Flight control, stabilization

In [5]:
# Simulate IMU quaternion readings
np.random.seed(42)

print('Drone IMU Data Processing\n')

# Quaternion from IMU (w, x, y, z format)
imu_quat = np.array([0.9239, 0.2209, 0.2209, 0.2209])  # ~45° roll
imu_quat = imu_quat / np.linalg.norm(imu_quat)  # Normalize

print(f'IMU quaternion: {imu_quat}\n')

# Convert to Rotation object (scipy uses x,y,z,w)
quat_scipy = np.array([imu_quat[1], imu_quat[2], imu_quat[3], imu_quat[0]])
r_drone = R.from_quat(quat_scipy)

# Extract Euler angles (aerospace convention: ZYX)
roll, pitch, yaw = r_drone.as_euler('xyz', degrees=True)

print('Drone orientation:')
print(f'  Roll:  {roll:.1f}° (rotation around X, forward axis)')
print(f'  Pitch: {pitch:.1f}° (rotation around Y, right axis)')
print(f'  Yaw:   {yaw:.1f}° (rotation around Z, up axis)\n')

# Determine if drone is level
if abs(roll) < 5 and abs(pitch) < 5:
    print('Status: LEVEL ✓')
else:
    print(f'Status: TILTED (roll={roll:.1f}°, pitch={pitch:.1f}°)')

Drone IMU Data Processing

IMU quaternion: [0.92390848 0.22090203 0.22090203 0.22090203]

Drone orientation:
  Roll:  32.1° (rotation around X, forward axis)
  Pitch: 18.1° (rotation around Y, right axis)
  Yaw:   32.1° (rotation around Z, up axis)

Status: TILTED (roll=32.1°, pitch=18.1°)


## Real Example: Robot Arm Kinematics
Compute end-effector position from joint angles
Forward kinematics using rotation composition

In [6]:
# Simple 2-joint robot arm
print('Robot Arm Forward Kinematics\n')

# Link lengths
L1 = 1.0  # meters
L2 = 0.8

# Joint angles
theta1 = 45  # degrees, base rotation
theta2 = 30  # degrees, elbow rotation

print(f'Configuration:')
print(f'  Link 1 length: {L1} m')
print(f'  Link 2 length: {L2} m')
print(f'  Joint 1 angle: {theta1}°')
print(f'  Joint 2 angle: {theta2}°\n')

# Base rotation
r1 = R.from_euler('z', theta1, degrees=True)
link1_end = r1.apply([L1, 0, 0])

print(f'Link 1 end position: {link1_end}')

# Elbow rotation (relative to link 1)
r2 = R.from_euler('z', theta2, degrees=True)
r_total = r1 * r2
link2_direction = r_total.apply([L2, 0, 0])

end_effector = link1_end + link2_direction
print(f'End effector position: {end_effector}\n')

# Workspace
max_reach = L1 + L2
min_reach = abs(L1 - L2)
print(f'Workspace:')
print(f'  Max reach: {max_reach} m')
print(f'  Min reach: {min_reach} m')
print(f'  Current reach: {np.linalg.norm(end_effector):.3f} m')

Robot Arm Forward Kinematics

Configuration:
  Link 1 length: 1.0 m
  Link 2 length: 0.8 m
  Joint 1 angle: 45°
  Joint 2 angle: 30°

Link 1 end position: [0.70710678 0.70710678 0.        ]
End effector position: [0.91416202 1.47984744 0.        ]

Workspace:
  Max reach: 1.8 m
  Min reach: 0.19999999999999996 m
  Current reach: 1.739 m


## Rotation Interpolation (SLERP)
Smoothly interpolate between two orientations
**SLERP**: Spherical Linear Interpolation

Applications: Animation, smooth camera motion

In [7]:
# Two orientations
r_start = R.from_euler('xyz', [0, 0, 0], degrees=True)
r_end = R.from_euler('xyz', [90, 45, 30], degrees=True)

print('Rotation Interpolation (SLERP)\n')
print(f'Start: {r_start.as_euler("xyz", degrees=True)}°')
print(f'End:   {r_end.as_euler("xyz", degrees=True)}°\n')

# Create interpolator
key_rots = R.from_quat([r_start.as_quat(), r_end.as_quat()])
key_times = [0, 1]

from scipy.spatial.transform import Slerp
slerp = Slerp(key_times, key_rots)

# Interpolate at intermediate times
times = [0.0, 0.25, 0.5, 0.75, 1.0]
print('Interpolated orientations:')
for t in times:
    r_interp = slerp(t)
    euler = r_interp.as_euler('xyz', degrees=True)
    print(f'  t={t:.2f}: [{euler[0]:6.1f}, {euler[1]:6.1f}, {euler[2]:6.1f}]°')

print('\nSmooth transition for animation!')

Rotation Interpolation (SLERP)

Start: [0. 0. 0.]°
End:   [90. 45. 30.]°

Interpolated orientations:
  t=0.00: [   0.0,    0.0,    0.0]°
  t=0.25: [  17.9,   14.0,   -0.8]°
  t=0.50: [  37.3,   27.4,    3.2]°
  t=0.75: [  60.8,   38.7,   13.2]°
  t=1.00: [  90.0,   45.0,   30.0]°

Smooth transition for animation!


## Summary

### Rotation Functions:
```python
from scipy.spatial.transform import Rotation as R, Slerp

# Create rotations
r = R.from_euler('xyz', [roll, pitch, yaw], degrees=True)
r = R.from_quat([x, y, z, w])
r = R.from_matrix(rotation_matrix)
r = R.from_rotvec(axis * angle)

# Convert representations
matrix = r.as_matrix()
quat = r.as_quat()
euler = r.as_euler('xyz', degrees=True)
rotvec = r.as_rotvec()

# Apply
rotated = r.apply(vectors)

# Compose
r_combined = r1 * r2

# Interpolate
slerp = Slerp(times, rotations)
r_interp = slerp(t)
```

### Euler Angle Conventions:
- **Intrinsic** (body-fixed): Rotations follow moving axes
- **Extrinsic** (space-fixed): Rotations follow fixed axes
- **Order matters**: 'xyz' ≠ 'zyx'
- **Aerospace**: Often ZYX (yaw-pitch-roll)

### Quaternion Advantages:
✓ No gimbal lock (unlike Euler angles)  
✓ Smooth interpolation (SLERP)  
✓ Compact (4 numbers vs 9 for matrix)  
✓ Efficient composition  

### Applications:
✓ **Robotics**: Joint angles, end-effector pose  
✓ **Drones/UAVs**: Attitude control, navigation  
✓ **Graphics**: 3D modeling, animation, games  
✓ **VR/AR**: Head tracking, controller orientation  
✓ **IMU**: Sensor fusion, AHRS  
✓ **Satellites**: Attitude determination  